# Post Earning Announcement Drift

Strategy is simple. We only trade the names where the earnings is announced after market close, the strategy assumpts the earnings announcement will draft the intraday momentum at the first trading day after earnings.

1. Souce all stocks where earnings announced at T-1 after market close or T market before market open

2. Calculate the T-1 close to T open returns $r_i$, we can proxy the T open using pre-open prices
$$r_i = \frac{open_t - close_{t-1}}{close_{t-1}}$$

3. For each name, compute the correlation T-1 close to T open returns vs Intraday return in next day among all earning release days

$$\beta = Correlation(\frac{open_t - close_{t-1}}{close_{t-1}}, \frac{close_t - open_t}{open_t})$$


4. If the z score of return exceeds certain threshold $z_{th}$
    - When $z_i > |z_{th}|$
        -  Long when $\beta > \beta_{th}$
        -  Short when $\beta < \beta_{th}$

    - When $z_i < -|z_{th}|$
        -  Short when $\beta > \beta_{th}$
        -  Long when $\beta < \beta_{th}$


5. Liquidate all positions by eod of day

The strategy starts by trading all stocks listed on SP500, but please be in mind that there is some survivorship bias by using currently listed SP500 stocks as our universe 

# Get all earnings calendar for backtest
- Get and store all earnigns in hdf5

In [18]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from utils.performance import *
from utils.data import *
from utils.earnings_calendar import *
from utils.data_helper import *
from utils.logging import get_logger
from pandas.tseries.offsets import BDay
from matplotlib.gridspec import GridSpec
from account.Futu import *
import warnings
from Strategy import PostEarningsAnnouncementDrift


pd.options.display.max_rows = 200
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

logger = get_logger('Post Earnings Announcement Drift')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# end_date = get_today() + BDay(30)
# start_date = end_date - BDay(252*10)

In [ ]:
# dates = pd.date_range(start_date, end_date)
# df_ec_arr = []
# with tqdm(total=len(dates)) as pbar:
#     for d in dates:
#         df_ec = get_earnings_by_from_investcom(d)
#         df_ec_arr.append(df_ec)
#         pbar.update(1)        

In [ ]:
# stock_universe = get_sp500_tickers()
# tickers_bucket = bucketize(stock_universe, 10)
# with tqdm(total=len(tickers)) as pbar:
#     for b in tickers_bucket:
#         get_earnings_by_symbols_from_yahoo(b)
#         pbar.update(len(b))        

# BackTest

1. Constrct a dataframe which contains all the historical stats (e.g. OCHL) on eanrings days

2. Iterate the stocks one by one and decide the trade signals on earnings release day

3. Combine the returns every day

4. Assume portfolios are equally weighted

### My Notes
- If we try announcemment mean reversion => seems profitable after 2021
- If we try announcement momentum        => seems profitable before 2021

In [21]:
start_date = datetime(2021,1,1)
end_date = add_bday(get_today(), -1)
stock_universe = get_sp500_tickers()

In [22]:
strategy = PostEarningsAnnouncementDrift()    
strategy.set_stock_universe(stock_universe)
strategy.set_start_date(start_date)
strategy.set_end_date(end_date)
strategy.preprocess_data()
strategy.generate_position()
strategy.backtest_summary()

2023-12-09 00:05:38,305 - Post Earnings Announcement Drift - INFO - ============================== Post Earnings Announcement Drift Setup ==============================
2023-12-09 00:05:38,306 - Post Earnings Announcement Drift - INFO - capital:                     20000
2023-12-09 00:05:38,307 - Post Earnings Announcement Drift - INFO - is_backtest:                 True
2023-12-09 00:05:38,307 - Post Earnings Announcement Drift - INFO - open_period_to_use_close:    20
2023-12-09 00:05:38,308 - Post Earnings Announcement Drift - INFO - earnings_lookback:           -1
2023-12-09 00:05:38,309 - Post Earnings Announcement Drift - INFO - z_threshold:                 0.5
2023-12-09 00:05:38,309 - Post Earnings Announcement Drift - INFO - beta_threshold:              0.3
2023-12-09 00:05:38,310 - Post Earnings Announcement Drift - INFO - min_trades:                  5
2023-12-09 00:05:38,310 - Post Earnings Announcement Drift - INFO - rolling_windows:             60
2023-12-09 00:05:38,310 -

2023-12-09 00:05:38,517 - Post Earnings Announcement Drift - INFO - Stock Universe: 503
2023-12-09 00:05:38,518 - Post Earnings Announcement Drift - INFO - Stored Names: 501
2023-12-09 00:05:38,518 - Post Earnings Announcement Drift - INFO - Missing Symbols: ['BF-B' 'BRK-B']


[*********************100%***********************]  503 of 503 completed


100%|██████████| 501/501 [00:01<00:00, 296.33it/s]
2023-12-09 00:05:57,920 - Post Earnings Announcement Drift - INFO - Generating positions.....
 98%|█████████▊| 492/501 [00:04<00:00, 104.72it/s]2023-12-09 00:06:02,827 - Post Earnings Announcement Drift - ERROR - Error calculating beta: CARR, x and y must have length at least 2.
2023-12-09 00:06:02,839 - Post Earnings Announcement Drift - ERROR - Error calculating beta: OTIS, x and y must have length at least 2.
2023-12-09 00:06:02,850 - Post Earnings Announcement Drift - ERROR - Error calculating beta: ABNB, x and y must have length at least 2.
2023-12-09 00:06:02,852 - Post Earnings Announcement Drift - ERROR - Error calculating beta: ABNB, x and y must have length at least 2.
2023-12-09 00:06:02,866 - Post Earnings Announcement Drift - ERROR - Error calculating beta: CEG, x and y must have length at least 2.
2023-12-09 00:06:02,869 - Post Earnings Announcement Drift - ERROR - Error calculating beta: CEG, x and y must have length at 

,Post Earnings Announcement Drift,^SPX,^IXIC
Measure,,,
Cumulative Return,0.918278,1.239131,1.129271
Annualized Return,-0.022770,0.088881,0.069466
Annualized Volatility,0.112634,0.176880,0.236320
Annualized Sharpe Ratio,-0.514665,0.303499,0.145004
Maximum Drawdown,-0.176138,-0.254251,-0.363953


In [ ]:
def plot_z_vs_next(df, quarter):    
    fig = plt.figure(figsize=(8,4))    
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    fig.suptitle(quarter, y=1.05, fontsize=20)
    gs = GridSpec(1,2)

    ax = fig.add_subplot(gs[0])
    x = df['z'].values
    y = df['intra_ret'].values

    ax.scatter(x, y)
    ax.axvline(0, linestyle='--', color='black')
    ax.axhline(0, linestyle='--', color='black')
    ax.plot(x, np.polyval(np.polyfit(x, y, 1),x), color='black', linewidth=5)
    ax.set_title('Z vs Next Intraday Return')
    ax.grid()
    ax.set_xlabel('Z Score')
    ax.set_ylabel('Next IntraDay Return')

    ax = fig.add_subplot(gs[1])
    x = df['open_ret'].values
    y = df['intra_ret'].values

    ax.scatter(x, y)
    ax.plot(x, np.polyval(np.polyfit(x, y, 1),x), color='black', linewidth=5)
    ax.axvline(0, linestyle='--', color='black')
    ax.axhline(0, linestyle='--', color='black')
    ax.set_title('Prev C2O Ret vs Next Intraday O2C Ret')
    ax.grid()
    ax.set_xlabel('Prev C20 Return')
    ax.set_ylabel('Next IntraDay Return')
    plt.show()

df_trade_stats = strategy.df_trade_stats
beta_threshold = strategy.beta_threshold

temp = df_trade_stats[df_trade_stats['beta'] > beta_threshold]
temp = temp[temp['position'] != 0]
plot_z_vs_next(temp, f'All Time (Traded Names with Beta > {beta_threshold})')

temp = df_trade_stats[df_trade_stats['beta'] < -beta_threshold]
temp = temp[temp['position'] != 0]
plot_z_vs_next(temp, f'All Time (Traded Names with Beta < -{beta_threshold})')

# for q in sorted(df_stats['quarter'].unique(), reverse=True):
#     temp = df_stats[df_stats['quarter'] == q]
#     plot_z_vs_next(temp, q)

In [ ]:
# Get latest return (i.e. returns for yesterday)
df_trade_stats = strategy.df_trade_stats
latest = df_trade_stats[df_trade_stats['position_date'] == end_date]

ret = (latest['weight'] * latest['intra_ret']).sum()
latest['correct_dir'] = np.where(latest['position'] != 0, np.sign(latest['position']) == np.sign(latest['intra_ret']), np.nan)    
correct_dir_pct = latest['correct_dir'].sum() / (1 * (latest['position'] != 0).sum())
no_of_long = len(latest[latest['position'] > 0])
no_of_short = len(latest[latest['position'] < 0])

logger.info('Intra-day return on {} is {:.2f}%, pct of hit is {:.2f}%'.format(end_date.strftime('%Y-%m-%d'), ret*100, correct_dir_pct*100))
logger.info('# of Longs:  {:.0f}'.format(no_of_long))
logger.info('# of Shorts: {:.0f}'.format(no_of_short))
display(latest)

# Parameters calibration

- Calibrate the model parameters by maximize the sharpe of the strategy

In [ ]:
# start_date = datetime(2021,1,1)
# end_date = add_bday(get_today(), -1)
# stock_universe = get_sp500_tickers()

# strategy = PostEarningsAnnouncementDrift()    
# strategy.set_stock_universe(stock_universe)
# strategy.set_start_date(start_date)
# strategy.set_end_date(end_date)
# strategy.preprocess_data() 

# fig = plt.figure(figsize=(8,4))    
# fig.subplots_adjust(hspace=0.5, wspace=0.3)
# gs = GridSpec(1,2)

# beta = np.arange(0,1,0.1)
# z_th = np.arange(0,3,0.1)
# beta_sr = []
# z_th_sr = []

# for b in beta:
#     strategy.__init__()
#     strategy.set_beta_threshold(b)
#     strategy.generate_position()
#     strategy.generate_backtest_return()
#     port_ret = strategy.port_ret
#     beta_sr.append(annualized_sharpe_ratio(port_ret))    

# for z in z_th:    
#     strategy.__init__()
#     strategy.set_z_threshold(z)    
#     strategy.generate_position()
#     strategy.generate_backtest_return()
#     port_ret = strategy.port_ret
#     z_th_sr.append(annualized_sharpe_ratio(port_ret))

# ax = fig.add_subplot(gs[0])
# ax.plot(beta, beta_sr)
# ax.set_ylabel('Sharpe Ratio')
# ax.set_xlabel('Beta threshold')
# ax.grid()
# ax.set_title('Beta threshold vs Sharpe')

# ax = fig.add_subplot(gs[1])
# ax.plot(z_th, z_th_sr)
# ax.set_ylabel('Sharpe Ratio')
# ax.set_xlabel('Z threshold')
# ax.grid()
# ax.set_title('Z threshold vs Sharpe')

# Execute the strategy

1. Get all earnings that is released after market at T-1 or before open at T => here we only get names with this constraints to speedup the time required to preprocess the data

2. Get all historical prices for these names

3. Get real-time open price

4. Follows the strategy to derive the positions

5. Convert the position to shares

6. Send the orders

7. Liquidate all orders by end of day

In [15]:
stock_universe = get_sp500_tickers()
capital = 20000
today = add_bday(add_bday(get_today(), -1), 1)

In [16]:
strategy = PostEarningsAnnouncementDrift()    
strategy.set_stock_universe()
strategy.set_capital(capital)
strategy.actual_trade(today)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1803    0  1701  100   102  11448    686 --:--:-- --:--:-- --:--:-- 12971
2023-12-08 22:53:43,379 - Earning Calendar - INFO - Fetched 11 names from Invest.com Earning Calendar: 2023-12-08 - 2023-12-08.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4489    0  4387  100   102  22506    523 --:--:-- --:--:-- --:--:-- 23502
2023-12-08 22:53:44,045 - Earning Calendar - INFO - Fetched 40 names from Invest.com Earning Calendar: 2023-12-07 - 2023-12-07.
2023-12-08 22:53:44,435 - Post Earnings Announcement Drift - INFO - ============================== Post Earnings Announcement Drift Setup ==============================
2023-12-08 22:53:44,435 - Post Earnings Announcement Drift - INFO - capital:                     2000

[*********************100%***********************]  3 of 3 completed

2023-12-08 22:53:45,004 - Post Earnings Announcement Drift - CRITICAL - Actual Trade Mode: After 20 mins of market open, use open price


100%|██████████| 3/3 [00:00<00:00, 154.76it/s]
2023-12-08 22:53:45,033 - Post Earnings Announcement Drift - INFO - Generating positions.....
100%|██████████| 3/3 [00:00<00:00, 241.80it/s]


In [17]:
today_stats = strategy.df_trade_stats

traded_ntl = today_stats['allocated_capital'].abs().sum()
no_of_long = len(today_stats[today_stats['position'] > 0])
no_of_short = len(today_stats[today_stats['position'] < 0])

logger.info('Total Traded Notional ${:,.0f}'.format(today_stats['allocated_capital'].abs().sum()))
logger.info('# of Longs:  {:.0f}'.format(no_of_long))
logger.info('# of Shorts: {:.0f}'.format(no_of_short))
today_stats

2023-12-08 22:53:45,121 - Post Earnings Announcement Drift - INFO - Total Traded Notional $0
2023-12-08 22:53:45,122 - Post Earnings Announcement Drift - INFO - # of Longs:  0
2023-12-08 22:53:45,123 - Post Earnings Announcement Drift - INFO - # of Shorts: 0


,symbol,date,d_prev,d_next,type,prev_close,next_open,next_close,z,rolling_mean,rolling_std,position_date,open_ret,intra_ret,quarter,beta,signal,weight,position,shares,allocated_capital
0,AVGO,2023-12-07,2023-12-07,2023-12-08,After market close,922.260010,905.539978,927.190002,-1.078910,0.001557,0.018247,2023-12-08,-0.018129,0.023908,2023Q4,0.138761,0,0.0,0.0,0.0,0.0
1,COO,2023-12-07,2023-12-07,2023-12-08,After market close,344.950012,342.809998,340.519989,-0.428725,0.000122,0.014754,2023-12-08,-0.006204,-0.006680,2023Q4,0.215220,0,0.0,0.0,0.0,0.0
2,LULU,2023-12-07,2023-12-07,2023-12-08,After market close,464.670013,460.670013,467.260010,-0.584074,0.003163,0.020153,2023-12-08,-0.008608,0.014305,2023Q4,0.226140,0,0.0,0.0,0.0,0.0


## Execute the orders to open the position

In [ ]:
strategy.enter_strategy(is_test=False)
strategy.execute_orders

## Schedule to close out all intraday positions

In [5]:
strategy.exit_strategy(exit_time='04:49:00', is_test=False)
strategy.exit_orders

2023-12-02 03:40:34,077 | 37014 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=18214795


2023-12-02 03:40:34,095 - Futu - INFO - 17 Positions: US.ZBH, US.WRK, US.ULTA, US.NSC, US.INCY, US.CRM, US.VOE, US.VBR, US.TSLA, US.SPY, US.QQQ, US.NVDA, US.KHC, US.EWY, US.BRK.B, US.AAPL


2023-12-02 03:40:34,097 | 37014 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2023-12-02 03:40:34,106 - Post Earnings Announcement Drift - INFO - Total 1 stocks to close....


2023-12-02 04:49:00,963 | 37014 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=18214795


...

2023-12-02 04:49:04,188 | 37014 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=4, host=127.0.0.1, port=11111, user_id=18214795


2023-12-02 04:49:04,454 - Futu - INFO - Placed Order: {'code': 'US.ULTA', 'price': 1, 'qty': 9.0, 'trd_side': 'SELL', 'order_type': 'MARKET', 'market': 'US', 'trd_env': 'REAL'}


2023-12-02 04:49:04,457 | 37014 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=4


2023-12-02 04:49:07,480 - Futu - INFO - Saving 2023-12-01 orders for strategy Post Earnings Announcement Drift....


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,dealt_qty,dealt_avg_price,last_err_msg,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,strategy,date
0,US.ULTA,Ulta美容,SELL,MARKET,SUBMITTING,2315046996427582354,9.0,0.0,2023-12-01 15:49:04,2023-12-01 15:49:04,0.0,0.0,,,DAY,False,N/A,N/A,N/A,N/A,USD,Post Earnings Announcement Drift,2023-12-01


# Order History

In [ ]:
Futu().order_history(start_date=today, end_date=today + BDay(1))